In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
#load the NSL_KDD dataset
train=pd.read_csv('dataset/KDDTrain+.csv')
test=pd.read_csv('dataset/KDDTest+.csv')

#size of train and test data
print(train.shape,test.shape)

# concatenate traina and test data
dataset=pd.concat([test,train])

#size of whole dataset
print(dataset.shape)

#print(dataset.head(2))
#print(dataset.columns)
X = dataset.iloc[:, :-1].values
print(X)
y = dataset.iloc[:, 41:42].values
print(y)

(113270, 42) (15451, 42)
(128721, 42)
[[0 'tcp' 'private' ... 0.0 1.0 1.0]
 [0 'tcp' 'private' ... 0.0 1.0 1.0]
 [2 'tcp' 'ftp_data' ... 0.0 0.0 0.0]
 ...
 [0 'tcp' 'smtp' ... 0.0 0.01 0.0]
 [0 'tcp' 'klogin' ... 1.0 0.0 0.0]
 [0 'tcp' 'ftp_data' ... 0.0 0.0 0.0]]
[['DDoS']
 ['DDoS']
 ['normal']
 ...
 ['normal']
 ['DDoS']
 ['normal']]


In [3]:
#Encoding of dataset
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X= ct.fit_transform(X)
print(X)

[[0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]
 ...
 [0.0 1.0 0.0 ... 0.0 0.01 0.0]
 [0.0 1.0 0.0 ... 1.0 0.0 0.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]]


In [4]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y[:, 0] = le_y.fit_transform(y[:, 0])
y= y.astype(int)
y = y.ravel()
print(y)

[0 0 1 ... 1 0 1]


In [5]:
# min_max normalisation between (a,b)
from sklearn.preprocessing import minmax_scale
X = minmax_scale(X, feature_range=(0,1), axis=0)
print(X)

[[0.   1.   0.   ... 0.   1.   1.  ]
 [0.   1.   0.   ... 0.   1.   1.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]
 ...
 [0.   1.   0.   ... 0.   0.01 0.  ]
 [0.   1.   0.   ... 1.   0.   0.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]]


In [6]:
# chi square
from sklearn.feature_selection import SelectKBest, chi2
selector2 = SelectKBest(chi2, k=15).fit(X, y)
cols = selector2.get_support(1)
XCS = X[:,cols]
print(XCS)

[[0.   1.   0.   ... 0.04 0.   0.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.61 0.   0.  ]
 ...
 [0.   0.   0.   ... 0.12 0.72 0.  ]
 [0.   0.   1.   ... 0.03 1.   1.  ]
 [0.   0.   0.   ... 0.3  0.   0.  ]]


In [7]:
print(XCS.shape)

(128721, 15)


In [8]:
# splitting of dataset in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XCS, y, test_size = 0.3, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[0.   0.   0.   ... 1.   0.   0.  ]
 [0.   0.   1.   ... 0.07 1.   1.  ]
 [0.   0.   0.   ... 0.61 0.   0.01]
 ...
 [1.   0.   0.   ... 1.   0.03 0.01]
 [1.   0.   0.   ... 1.   0.   0.  ]
 [1.   0.   0.   ... 1.   0.   0.  ]]
[[0.   0.   0.   ... 0.8  0.   0.  ]
 [0.   0.   1.   ... 0.26 1.   1.  ]
 [0.   0.   1.   ... 0.08 1.   1.  ]
 ...
 [0.   0.   0.   ... 0.49 0.79 0.79]
 [0.   0.   1.   ... 0.05 1.   1.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]]
[1 0 1 ... 1 1 1]
[1 0 0 ... 1 0 1]


In [9]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [10]:
# base classifier RandomForest
y_train = y_train.ravel()
clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred1)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE= ",RMSE)


population: 38617
P: 23198
N: 15419
PositiveTest: 23388
NegativeTest: 15229
TP: 22932
TN: 14963
FP: 456
FN: 266
TPR: 0.9885334942667471
TNR: 0.9704260976717037
PPV: 0.9805028219599795
NPV: 0.9825333245781076
FPR: 0.029573902328296257
FDR: 0.019497178040020522
FNR: 0.011466505733252867
ACC: 0.9813035709661548
F1_score: 0.9845017816511398
MCC: 0.9609957076418927
informedness: 0.9589595919384508
markedness: 0.9630361465380872
prevalence: 0.6007198902037962
LRP: 33.4258726931995
LRN: 0.011815949468758
DOR: 2828.8774238227143
FOR: 0.017466675421892442
RMSE=  0.13673488594299993


In [11]:
# base classifier GradientBoosting
clf2 = GradientBoostingClassifier(n_estimators=100)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred2)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE= ",RMSE)


population: 38617
P: 23198
N: 15419
PositiveTest: 23772
NegativeTest: 14845
TP: 23071
TN: 14718
FP: 701
FN: 127
TPR: 0.9945253901198379
TNR: 0.9545366106751411
PPV: 0.9705115261652364
NPV: 0.9914449309531829
FPR: 0.04546338932485894
FDR: 0.029488473834763586
FNR: 0.005474609880162083
ACC: 0.9785586658725431
F1_score: 0.9823717266340217
MCC: 0.9554874776104899
informedness: 0.9490620007949788
markedness: 0.9619564571184194
prevalence: 0.6007198902037962
LRP: 21.875302411209386
LRN: 0.005735358726879954
DOR: 3814.112325474294
FOR: 0.00855506904681711
RMSE=  0.146428597369014


In [12]:
# SVM
from sklearn.svm import SVC  
clf3= SVC(kernel='linear') 
clf3.fit(X_train, y_train) 
y_pred3 = clf3.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred3)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE= ",RMSE)


population: 38617
P: 23198
N: 15419
PositiveTest: 24061
NegativeTest: 14556
TP: 22948
TN: 14306
FP: 1113
FN: 250
TPR: 0.9892232088973187
TNR: 0.9278163305013295
PPV: 0.9537425709654628
NPV: 0.9828249519098653
FPR: 0.07218366949867047
FDR: 0.046257429034537216
FNR: 0.010776791102681265
ACC: 0.964704663749126
F1_score: 0.9711589326900696
MCC: 0.9267520972694498
informedness: 0.9170395393986484
markedness: 0.9365675228753281
prevalence: 0.6007198902037962
LRP: 13.704252163511013
LRN: 0.011615220327991223
DOR: 1179.8529667565138
FOR: 0.01717504809013465
RMSE=  0.18787053055461883


In [13]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
clf4 = KNeighborsClassifier(n_neighbors = 5)
clf4.fit(X_train, y_train) 
y_pred4 = clf4.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred4)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE= ",RMSE)


population: 38617
P: 23198
N: 15419
PositiveTest: 23369
NegativeTest: 15248
TP: 22923
TN: 14973
FP: 446
FN: 275
TPR: 0.9881455297870506
TNR: 0.9710746481613594
PPV: 0.9809148872437845
NPV: 0.9819648478488983
FPR: 0.028925351838640638
FDR: 0.019085112756215498
FNR: 0.011854470212949391
ACC: 0.9813294662972266
F1_score: 0.9845169325917495
MCC: 0.9610482146274042
informedness: 0.95922017794841
markedness: 0.9628797350926828
prevalence: 0.6007198902037962
LRP: 34.161919111629
LRN: 0.012207578722598455
DOR: 2798.4189074602527
FOR: 0.018035152151101783
RMSE=  0.13664016138300406


In [14]:
# meta classifier LogisticRegression
lr = LogisticRegression(solver='lbfgs')

# stack esemble method 
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2,clf4], 
                          meta_classifier=lr)

In [15]:
# train and test the model
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE= ",RMSE)


population: 38617
P: 23198
N: 15419
PositiveTest: 23327
NegativeTest: 15290
TP: 22909
TN: 15001
FP: 418
FN: 289
TPR: 0.9875420294853005
TNR: 0.972890589532395
PPV: 0.9820808505165688
NPV: 0.9810987573577502
FPR: 0.027109410467604903
FDR: 0.017919149483431217
FNR: 0.012457970514699544
ACC: 0.981692000932232
F1_score: 0.9848038688876948
MCC: 0.9618051327452818
informedness: 0.9604326190176955
markedness: 0.9631796078743191
prevalence: 0.6007198902037962
LRP: 36.42801567615753
LRN: 0.0128051094837779
DOR: 2844.8031406764785
FOR: 0.018901242642249835
RMSE=  0.1353070547597873
